# MODULO_05 FINE-TUNING COMPLETO DEL MODELO LOGFORMER

# ---------------------------------------------
# ATENCIÓN - FIJAR ESTAS VARIABLES ANTES DE EJECUTAR
# ---------------------------------------------

In [1]:
# 🛠 Instalar librerías necesarias en Colab (transformers + accelerate actualizados)
%pip install -q --upgrade "transformers[torch]" "accelerate>=0.26.0"


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 113.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 92.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 54.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 36.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 105.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 129.5 MB/s eta 0:00:00


In [2]:
import transformers
print("Transformers version:", transformers.__version__)


Transformers version: 4.53.0


In [1]:
nombre_lote = "LOTE_20250614"
nombre_modulo = "MODULO_05"


In [2]:
# ---------------------------------------------
# Configuración del entorno (Colab y Local)
# ---------------------------------------------

try:
    import google.colab
    EN_COLAB = True
except ImportError:
    EN_COLAB = False

if EN_COLAB:
    from google.colab import drive
    drive.mount("/content/drive", force_remount=True)
    ruta_base = "/content/drive/MyDrive/TFM_EVA_MARTIN/Modulos"
else:
    ruta_base = "G:/Mi unidad/TFM_EVA_MARTIN/Modulos"

print(f"Entorno detectado: {'Google Colab' if EN_COLAB else 'Local'}")
print(f"Ruta base: {ruta_base}")

lote_id = nombre_lote.replace("LOTE_", "")

Entorno detectado: Local
Ruta base: G:/Mi unidad/TFM_EVA_MARTIN/Modulos


In [3]:
import sys
import os
ruta_config = os.path.join(ruta_base, "config.yaml")

if ruta_base not in sys.path:
    sys.path.append(ruta_base)
import yaml

# Cargar configuración desde el archivo YAML
with open(ruta_config, "r", encoding="utf-8") as f:
    yaml_cfg = yaml.safe_load(f)   # ← renombrado

# Extraer bloque de parámetros (KeyError si falta alguna clave)
params = yaml_cfg["parametros"]


# Carga utilidades comunes e inicialización del entorno

In [4]:
import pandas as pd

import utilidades_comunes

# 1. Configurar logger
import logging
logger = utilidades_comunes.configurar_logger(nombre_modulo, ruta_logs=os.path.join(ruta_base, nombre_modulo, "logs"))
logger.setLevel(logging.DEBUG)

# 2. Inicializar entorno
entorno = utilidades_comunes.inicializar_entorno(nombre_modulo, nombre_lote, ruta_base, ruta_config, logger=logger)

2025-06-28 15:21:53,369 - INFO - 📁 Entorno inicializado para MODULO_05
2025-06-28 15:21:53,920 - INFO - 📂 Ruta entrada: G:/Mi unidad/TFM_EVA_MARTIN\Modulos\MODULO_04\./salida
2025-06-28 15:21:53,921 - INFO - 📂 Ruta salida: G:/Mi unidad/TFM_EVA_MARTIN/Modulos\MODULO_05\./salida
2025-06-28 15:21:53,921 - INFO - 📂 Ruta logs: G:/Mi unidad/TFM_EVA_MARTIN/Modulos\MODULO_05\./logs
2025-06-28 15:21:53,921 - INFO - 📂 Ruta ejemplos: G:/Mi unidad/TFM_EVA_MARTIN/Modulos\MODULO_05\./ejemplos
2025-06-28 15:21:53,921 - INFO - 🔗 Módulo anterior: MODULO_04
2025-06-28 15:21:53,921 - INFO - 🆔 Lote ID: 20250614


# Carga y analisis dataset de entrada

In [5]:
patron_busqueda = os.path.join(
    entorno["ruta_entrada"],
    f"dataset_{entorno['nombre_modulo_anterior'].lower()}_{entorno['lote_id']}*.csv"
)

import glob
archivos_encontrados = glob.glob(patron_busqueda)

if not archivos_encontrados:
    raise FileNotFoundError(f"No se encontró archivo de entrada para el lote {nombre_lote} con patrón: {patron_busqueda}")

fichero_entrada = archivos_encontrados[0]
df_entrada = utilidades_comunes.cargar_dataset(fichero_entrada, logger=logger)

utilidades_comunes.mostrar_muestra_dataset(df_entrada, "dataset de entrada", logger=logger)
utilidades_comunes.guardar_muestra_dataset(df_entrada, "entrada", entorno["ruta_ejemplos"], logger=logger, n=5)


df = df_entrada.copy()
# ---------------------------------------------
# 3. Cargar datasets ya divididos del módulo anterior
# ---------------------------------------------

# Rutas de los archivos divididos
ruta_train = os.path.join(entorno["ruta_entrada"], "train.csv")
ruta_val = os.path.join(entorno["ruta_entrada"], "val.csv")
ruta_test = os.path.join(entorno["ruta_entrada"], "test.csv")

# Cargar los datasets
df_train = utilidades_comunes.cargar_dataset(ruta_train, logger=logger)
df_val = utilidades_comunes.cargar_dataset(ruta_val, logger=logger)
df_test = utilidades_comunes.cargar_dataset(ruta_test, logger=logger)

# Verificar que los datasets se cargaron correctamente
logger.info(f"✅ Dataset train cargado desde {ruta_train} ({len(df_train)} filas)")
logger.info(f"✅ Dataset val cargado desde {ruta_val} ({len(df_val)} filas)")
logger.info(f"✅ Dataset test cargado desde {ruta_test} ({len(df_test)} filas)")

# Mostrar muestra de cada dataset
utilidades_comunes.mostrar_muestra_dataset(df_train, "dataset de entrenamiento", logger=logger)
utilidades_comunes.mostrar_muestra_dataset(df_val, "dataset de validación", logger=logger)
utilidades_comunes.mostrar_muestra_dataset(df_test, "dataset de prueba", logger=logger)

# Guardar muestras en la carpeta de ejemplos
utilidades_comunes.guardar_muestra_dataset(df_train, "train", entorno["ruta_ejemplos"], logger=logger, n=5)
utilidades_comunes.guardar_muestra_dataset(df_val, "val", entorno["ruta_ejemplos"], logger=logger, n=5)
utilidades_comunes.guardar_muestra_dataset(df_test, "test", entorno["ruta_ejemplos"], logger=logger, n=5)

# Copia para trabajar con los datos
df = df_train.copy()  # Usa df_train como base si necesitas un df único para alguna operación

2025-06-28 15:22:01,233 - INFO - ✅ Dataset cargado desde G:/Mi unidad/TFM_EVA_MARTIN\Modulos\MODULO_04\./salida\dataset_modulo_04_20250614.csv (140 filas, 4 columnas)
2025-06-28 15:22:01,233 - INFO - --- Muestra de dataset de entrada (primeras 5 filas) ---
2025-06-28 15:22:01,251 - INFO - Filas totales: 140, Columnas totales: 4
2025-06-28 15:22:01,461 - INFO - 
| nomfichero                                                                                            | etiqueta   | transcripcion                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        

In [6]:
# ---------------------------------------------
# 2. Cargar modelo y tokenizador ya ajustados
# ---------------------------------------------
from transformers import AutoConfig, LongformerTokenizerFast, LongformerForSequenceClassification
import torch

modelo_dir = os.path.join(entorno["ruta_entrada"], "modelo_ajustado")

# 1) Tokenizer con vocab actualizado
tokenizer = LongformerTokenizerFast.from_pretrained(modelo_dir)

# 2) AutoConfig para leer el config.json (incluye num_labels=3)
hf_config = AutoConfig.from_pretrained(modelo_dir)
print("✅ num_labels en config HF:", hf_config.num_labels)

# 3) Modelo con el config correcto
model = LongformerForSequenceClassification.from_pretrained(
    modelo_dir,
    config=hf_config
)

logger.info(f"📥 Modelo y tokenizador cargados desde: {modelo_dir}")

# 4) Descongelar todas las capas
for param in model.parameters():
    param.requires_grad = True

✅ num_labels en config HF: 3


2025-06-28 15:23:42,139 - INFO - 📥 Modelo y tokenizador cargados desde: G:/Mi unidad/TFM_EVA_MARTIN\Modulos\MODULO_04\./salida\modelo_ajustado


In [14]:
# Tokenización

class TFM_Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_texts = df_train["texto_etiquetado"].tolist()
train_labels = df_train["label"].tolist()
train_encodings = tokenizer(train_texts, truncation=True, padding=True, return_tensors="pt")
train_dataset = TFM_Dataset(train_encodings, train_labels)


if not df_val.empty:
    val_texts = df_val["texto_etiquetado"].tolist()
    val_labels = df_val["label"].tolist()
    val_encodings = tokenizer(val_texts, truncation=True, padding=True, return_tensors="pt")
    val_dataset = TFM_Dataset(val_encodings, val_labels)
else:
    val_dataset = None

if not df_test.empty:
    test_texts = df_test["texto_etiquetado"].tolist()
    test_labels = df_test["label"].tolist()
    test_encodings = tokenizer(test_texts, truncation=True, padding=True, return_tensors="pt")
    test_dataset = TFM_Dataset(test_encodings, test_labels)
else:
    test_dataset = None

In [ ]:
# ---------------------------------------------
# 4. Fine-Tuning completo
# ---------------------------------------------
from transformers import Trainer, TrainingArguments, AutoConfig

training_args = TrainingArguments(
    output_dir=os.path.join(entorno["ruta_salida"], "checkpoints"),
    num_train_epochs=4,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    eval_strategy="epoch", # Corrected argument name
    save_strategy="epoch",
    logging_dir=os.path.join(entorno["ruta_logs"]),
    logging_steps=10,
    save_total_limit=2,
    load_best_model_at_end=True,
    report_to="none",
    metric_for_best_model="accuracy"
)
def compute_metrics(eval_pred):
    from sklearn.metrics import accuracy_score, precision_recall_fscore_support
    logits, labels = eval_pred
    predictions = torch.argmax(torch.tensor(logits), dim=-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average='weighted')
    acc = accuracy_score(labels, predictions)
    return {"accuracy": acc, "precision": precision, "recall": recall, "f1": f1}

# Load model configuration first
config = AutoConfig.from_pretrained(modelo_dir, num_labels=3)
model = LongformerForSequenceClassification.from_pretrained(modelo_dir, config=config)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    # test_dataset=test_dataset,
    tokenizer=tokenizer,         # ← aquí
    compute_metrics=compute_metrics
)


logger.info("🚀 Iniciando Fine-Tuning completo del modelo…")
trainer.train()

/tmp/ipython-input-12-1327801729.py:32: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
2025-06-28 09:29:09,367 - INFO - 🚀 Iniciando Fine-Tuning completo del modelo…
INFO:MODULO_05:🚀 Iniciando Fine-Tuning completo del modelo…
Initializing global attention on CLS token...
Input ids are automatically padded to be a multiple of `config.attention_window`: 512


Epoch,Training Loss,Validation Loss


RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [ ]:
# ---------------------------------------------
# 5. Guardar modelo final fine-tuned
# ---------------------------------------------
modelo_final_dir = os.path.join(entorno["ruta_salida"], "modelo_final")
model.save_pretrained(modelo_final_dir)
tokenizer.save_pretrained(modelo_final_dir)
logger.info(f"✅ Fine-Tuning completado. Modelo guardado en: {modelo_final_dir}")


In [ ]:
%pip install tabulate